In [3]:
# Importing the necessary libraries
import os
import numpy as np
from skimage import io, morphology, filters
from skimage.io import imsave
from skimage.measure import label, regionprops
from skimage.morphology import reconstruction
from skimage.util import img_as_float
from scipy.signal import wiener

In [ ]:
# Defining the directory where the MRI images are located
mri_images_dir = "/Inputs/MRI Images/"
mri_file_list = os.listdir(mri_images_dir)

# Defining the output directories
sat_output_path = "/Outputs Threshold/SAT Masks/"
vat_output_path = "/Outputs Threshold/VAT Masks/"
total_output_path = "/Outputs Threshold/Total Masks/"

# Ensuring that the directories exist, if not, we create them
os.makedirs(sat_output_path, exist_ok=True)
os.makedirs(vat_output_path, exist_ok=True)
os.makedirs(total_output_path, exist_ok=True)

# Loop through each MRI image
for mri_file in mri_file_list:
    # Forming a complete filepath for each MRI image
    mri_file_path = os.path.join(mri_images_dir, mri_file)

    # Load the MRI image
    mri_img = img_as_float(io.imread(mri_file_path, as_gray=True))

    # Apply Wiener filter for noise reduction
    wiener_window_size = [6, 6]
    filtered_img = wiener(mri_img, wiener_window_size)

    # Perform morphological operations: Top-Hat and Bottom-Hat
    disk_elem = morphology.disk(50)
    white_tophat_img = morphology.white_tophat(filtered_img, disk_elem)
    black_tophat_img = morphology.black_tophat(filtered_img, disk_elem)

    # Enhance the image
    enhanced_img = mri_img + white_tophat_img - black_tophat_img
    enhanced_img = np.clip(enhanced_img, 0, 1)

    # Perform contrast enhancement
    contrast_enhanced_img = enhanced_img - black_tophat_img
    contrast_enhanced_img = np.clip(contrast_enhanced_img, 0, 1)

    # Find the optimal threshold value using Otsu's method
    otsu_threshold = filters.threshold_otsu(contrast_enhanced_img)

    # Apply threshold to get binary image
    binary_img = contrast_enhanced_img > otsu_threshold
    binary_img = img_as_float(binary_img)

    segmented_img = binary_img

    # Label connected components in the binary image
    labeled_img, num_obj = label(binary_img, connectivity=2, return_num=True)
    obj_props = regionprops(labeled_img)

    # Initialize SAT and VAT images
    sat_image = binary_img.copy()
    vat_image = binary_img.copy()

    # Compute area and centroids for all regions
    region_areas = [prop.area for prop in obj_props]
    region_centroids = [prop.centroid for prop in obj_props]

    # Compute the center of the image
    img_center = np.array(binary_img.shape) / 2

    # Calculate Euclidean distances of each object's centroid to the center of the image
    centroid_distances = [np.linalg.norm(np.array(centroid) - img_center) for centroid in region_centroids]

    # Find the indices of the two largest areas
    sorted_area_indices = np.argsort(region_areas)[::-1]

    # Determine whether the first or second-largest area is closer to the center of the image
    distance_first = centroid_distances[sorted_area_indices[0]]
    distance_second = centroid_distances[sorted_area_indices[1]]

    # Assign the labels for SAT and VAT based on their relative positions
    if distance_first < distance_second:
        sat_label = sorted_area_indices[0] + 1
        vat_label = sorted_area_indices[1] + 1
    else:
        sat_label = sorted_area_indices[1] + 1
        vat_label = sorted_area_indices[0] + 1

    # Create images with the SAT and VAT areas
    sat_image[labeled_img != sat_label] = 0
    vat_image[labeled_img != vat_label] = 0

    # Create threshold image for arm removal
    threshold_value = 0.1
    binary_img = mri_img > threshold_value

    # Remove arms from the image
    arm_removal_seed = np.copy(binary_img)
    arm_removal_seed[1:-1, 1:-1] = binary_img.max()
    binary_img = reconstruction(arm_removal_seed, binary_img, method='erosion')

    # Label the image for arm removal
    arm_removed_labeled_img = label(binary_img)
    arm_removed_areas = regionprops(arm_removed_labeled_img)

    # Identify the region with maximum area
    max_area = max(area['area'] for area in arm_removed_areas)
    max_area_pos = [area.label for area in arm_removed_areas if area['area'] == max_area]

    # Get the binary image with arms removed
    arm_removed_img = np.isin(arm_removed_labeled_img, max_area_pos)

    mri_img_without_arms = arm_removed_img
    sat_cmma_image = sat_image

    # Prepare total image
    morphological_square = morphology.square(10)
    mri_img_without_arms = morphology.erosion(mri_img_without_arms, morphological_square)

    segmented_img[mri_img_without_arms == 0] = 0
    sat_cmma_image_inverted = 1 - sat_cmma_image
    segmented_img[sat_cmma_image_inverted == 0] = 0

    total_img = segmented_img + sat_cmma_image

    # Save SAT, VAT, and Total images
    imsave(sat_output_path + mri_file, (sat_cmma_image * 255).astype(np.uint8))
    imsave(vat_output_path + mri_file, (segmented_img * 255).astype(np.uint8))
    imsave(total_output_path + mri_file, (total_img * 255).astype(np.uint8))